In [1]:
using JLD
using Convex, SCS # for optimization
using ClobberingReload # for deprecation warnings supression

In [2]:
# this contains some helper functions for MIL manipulation
include("/home/vit/Dropbox/vyzkum/cisco/kod/lib/julia/VBMatrixFactorization/examples/mil_util.jl")

plot_statistics

In [3]:
abstract type kernel
end

In [ ]:
struct quadkernel <: kernel
    c # offset
    km # kernel matrix

In [6]:
struct polykernel <: kernel
    p # polynomial order
    c # offset
    x # the data
    km # kernel matrix
    ϕ # generating function
end

# abbreviated constructor
polykernel(X::AbstractArray, p::Int, c = 0.0) = 
     polykernel(p, ones(size(X,1))*c, X, (X*X').^p, nothing)

# overloaded for direct calling
(pk::polykernel)(x) = (x'*x + c)^(pk.p)

In [5]:
X = rand(3,6)
krnl = polykernel(X,2)

polykernel(2, [0.0238552 0.29988 … 0.855878 0.301596; 0.851979 0.939919 … 0.929722 0.242266; 0.552347 0.137169 … 0.976116 0.76581], [2.1257 1.50628 2.70062; 1.50628 6.44282 3.05084; 2.70062 3.05084 5.59467], nothing)

In [9]:
# kernelized smallest hypersphere for anomaly detection
mutable struct kernel_sh
    k::kernel
    α
    r
    D
    c
    opt
end

# constructor
kernel_sh(k::kernel) = kernel_sh(k, Array{Float64,1}([]), NaN, NaN, NaN, false)

kernel_sh

In [10]:
ksh = kernel_sh(krnl)

kernel_sh(polykernel(2, [0.0238552 0.29988 … 0.855878 0.301596; 0.851979 0.939919 … 0.929722 0.242266; 0.552347 0.137169 … 0.976116 0.76581], [2.1257 1.50628 2.70062; 1.50628 6.44282 3.05084; 2.70062 3.05084 5.59467], nothing), Float64[], NaN, NaN, NaN, false)

In [11]:
W(α,gm) = diag(gm)'*α - sum(gm.*(α*α'))
W(ksh::kernel_sh) = W(ksh.α, ksh.k.gm)

function find_sphere!(ksh::kernel_sh)
    # this is the optimization part using Convex and SCS packages
    # setup the variables, objective and constraints
    n = size(ksh.k.gm,1)
    α = Variable(n, Positive())
    dgm = diag(ksh.k.gm)'
    dg = ksh.k.gm
    objective = dgm*α - quadform(α, dg)
    constraints = [sum(α) == 1.0]
    problem = maximize(objective, constraints) 
    # solve
    no_warnings() do
        solve!(problem)
    end
    # if succesfully solved, extract and round the optimal solution
    if problem.status == :Optimal
        ksh.α = round.(problem.solution.primal[1:n], 3)
        ksh.opt = true
    else
        error("solution not found!")
    end
    
    # compute the radius
    ksh.r = sqrt(W(ksh))
    # compute D for the anomaly score function
    ksh.D = ksh.α'*ksh.k.gm*ksh.α - ksh.r^2
    # compute the center - but we don't really need this
    # also, it may not be possible for some kernels
    if !(ksh.k.ϕ == nothing)
        ksh.c = ksh.α'*ksh.k.ϕ.(ksh.k.x)
    end
end

function anomaly_score(ksh::kernel_sh, x)
    return ksh.k
end

LoadError: [91msyntax: incomplete: "function" at In[11]:37 requires end[39m

In [23]:
find_sphere!(ksh)

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 24
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 6, constraints m = 13
Cones:	primal zero / dual free vars: 2
	linear vars: 4
	soc vars: 7, soc blks: 2
Setup time: 4.29e-05s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  2.28e-05 
   100| 1.34e-03  2.28e-02  1.92e-02 -8.82e-01 -9.36e-01  7.39e-17  6.60e-05 
   200| 4.64e-05  6.51e-04  5.01e-04 -8.93e-01 -8.95e-01  7.35e-17  1.07e-04 
   240| 1.49e-05  3.91e-05  4.08e-

In [26]:
ksh.D

0.5168458726337386